In [12]:
import json
import re
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
def vinted_session():
    s = cloudscraper.create_scraper()
    s.headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'fr',
        'DNT': '1',
        'Connection': 'keep-alive',
        'TE': 'Trailers',
    }
    req = s.get("https://www.vinted.fr/")
    csrf_token_index = req.text.find("CSRF_TOKEN")
    start_index = csrf_token_index + len("CSRF_TOKEN\":\"")
    end_index = req.text.find("\",", start_index)
    csrf_token_value = req.text[start_index:end_index]


    s.headers['X-CSRF-Token'] = csrf_token_value
    return s

In [14]:
def scrapping_user(s, idUser, annonce_id):

    item_titles_advice, ratings_advice, feedbacks_advice = [], [], []
    item_id_dressing, item_title_dressing, item_brand_dressing = [], [], []
    item_size_dressing, item_price_dressing, item_created_dressing,item_description_dressing = [], [], [],[]

    try:
        url = f"https://www.vinted.fr/api/v2/users/{idUser}?localize=false"
        check = s.get(url)
        check.raise_for_status()
        json_data = json.loads(check.text)

        if json_data['user']['photo'] is None:
            photo = 'Null'
        else:
            photo = json_data['user']['photo']['url']
        
        if json_data['user']['business_account'] is None:
            business=False
        else:
            business=True

        if json_data['user']['feedback_count'] > 0:
            url_advice = f"https://www.vinted.fr/api/v2/user_feedbacks?user_id={idUser}&page=1&per_page={json_data['user']['feedback_count']}&by=all"
            check_advice = s.get(url_advice)
            check_advice.raise_for_status()
            json_data_advice = json.loads(check_advice.text)
            for advice in json_data_advice['user_feedbacks']:
                item_titles_advice.append(advice['item_title'])
                ratings_advice.append(advice['rating'])
                feedbacks_advice.append(advice['feedback'])
            time.sleep(1)  # Attendre 1 seconde avant le prochain appel API

        url_dressing = f"https://www.vinted.fr/api/v2/users/{idUser}/items?page=1&per_page={json_data['user']['item_count']}&order=relevance"
        check_dressing = s.get(url_dressing)
        check_dressing.raise_for_status()
        json_data_dressing = json.loads(check_dressing.text)

        for item in json_data_dressing["items"]:
            if item['id'] == annonce_id:
                annonce_date = item['created_at_ts']
                annonce_description = item['description']
            else:
                item_id_dressing.append(item['id'])
                item_title_dressing.append(item['title'])
                item_brand_dressing.append(item['brand'])
                item_size_dressing.append(item['size'])
                item_price_dressing.append(item['original_price_numeric'])
                item_created_dressing.append(item['created_at_ts'])
                item_description_dressing.append(item['description'])
        time.sleep(1)  # Attendre 1 seconde avant le prochain appel API

        

        return (json_data['user']['login'], json_data['user']['email'], json_data['user']['gender'], json_data['user']['item_count'],
                  json_data['user']['followers_count'], json_data['user']['following_count'],
                  json_data['user']['feedback_count'], json_data['user']['positive_feedback_count'],
                  json_data['user']['neutral_feedback_count'], json_data['user']['negative_feedback_count'],
                  json_data['user']['account_ban_date'], json_data['user']['is_account_ban_permanent'],
                  json_data['user']['is_publish_photos_agreed'], json_data['user']['expose_location'],
                  json_data['user']['city'], json_data['user']['country_title'], business,
                  json_data['user']['total_items_count'], json_data['user']['verification']['email']['valid'],
                  json_data['user']['verification']['facebook']['valid'],
                  json_data['user']['verification']['google']['valid'],
                  json_data['user']['verification']['phone']['valid'], json_data['user']['is_hated'],
                  json_data['user']['hates_you'], json_data['user']['can_view_profile'], item_titles_advice,
                  ratings_advice, feedbacks_advice, item_id_dressing, item_title_dressing, item_brand_dressing,
                  item_size_dressing, item_price_dressing, item_created_dressing, item_description_dressing, photo, annonce_date, annonce_description)

    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        return None


In [15]:
def scrapping_annonces(dataset_final, legend, s):
    try:
        url = f"https://www.vinted.fr/api/v2/catalog/items?page=1&per_page=800&search_text={legend}&catalog_ids=&color_ids=&brand_ids=&size_ids=&material_ids=&video_game_rating_ids=&status_ids=6"
        check = s.get(url)
        check.raise_for_status()
        json_data = json.loads(check.text)

        for annonce in json_data['items']:
            try:
                
                (login, email, gender, item_count, followers_count, following_count,
                feedback_count, positive_feedback_count, neutral_feedback_count,
                negative_feedback_count, account_ban_date, is_account_ban_permanent,
                is_publish_photos_agreed, expose_location, city, country_title, business_account,
                total_items_count, verification_email, verification_facebook, verification_google,
                verification_phone, is_hated, hates_you, can_view_profile, item_titles_advice,
                ratings_advice, feedbacks_advice, item_id_dressing, item_title_dressing,
                item_brand_dressing, item_size_dressing, item_price_dressing, item_created_dressing,
                item_description_dressing, photo, date_advert, description_advert) = scrapping_user(s, annonce['user']['id'], annonce['id'])
                dataset_annonce = pd.DataFrame(
                    [[annonce['id'], annonce['title'], annonce['price'], annonce['brand_title'], annonce['status'],annonce['url'],annonce['promoted'],
                      annonce['size_title'], annonce['favourite_count'], annonce['photo']['url'], date_advert, description_advert,
                      annonce['user']['id'],login, email, gender, item_count, followers_count, following_count,
                      feedback_count, positive_feedback_count, neutral_feedback_count,
                      negative_feedback_count, account_ban_date, is_account_ban_permanent,
                      is_publish_photos_agreed, expose_location, city, country_title, business_account,
                      total_items_count, verification_email, verification_facebook, verification_google,
                      verification_phone, is_hated, hates_you, can_view_profile, item_titles_advice,
                      ratings_advice, feedbacks_advice, item_id_dressing, item_title_dressing,
                      item_brand_dressing, item_size_dressing, item_price_dressing, item_created_dressing,
                      item_description_dressing, photo]],
                    columns=['id_advert', 'title_advert', 'price', 'brand', 'etat','url_advert','promoted', 'taille', 'favourite_count', 'photo_advert', 'date_advert',
                             'description_advert', 'idUser','login','email', 'gender', 'item_count', 'followers_count', 'following_count',
                     'feedback_count', 'positive_feedback_count', 'neutral_feedback_count',
                     'negative_feedback_count', 'account_ban_date', 'is_account_ban_permanent',
                     'is_publish_photos_agreed', 'expose_location', 'city', 'country_title', 'business_account',
                     'total_items_count', 'verification_email', 'verification_facebook', 'verification_google',
                     'verification_phone', 'is_hated', 'hates_you', 'can_view_profile', 'item_titles_advice',
                     'ratings_advice', 'feedbacks_advice', 'item_id_dressing', 'item_title_dressing',
                     'item_brand_dressing', 'item_size_dressing', 'item_price_dressing', 'item_created_dressing',
                     'item_description_dressing', 'photo']
                )
                dataset_final = pd.concat([dataset_final, dataset_annonce], axis=0, ignore_index=True)
            except Exception as e_annonce:
                print(f"Une erreur s'est produite lors du scraping d'une annonce : {e_annonce}")
            
            time.sleep(1)  # Attendre 1 seconde avant le prochain appel API pour les annonces
        
        print(dataset_final.shape)
        return dataset_final

    except Exception as e:
        print(f"Une erreur s'est produite lors du scraping des annonces : {e}")
        return dataset_final

In [16]:
dataset_final=pd.DataFrame(columns=['id_advert', 'title_advert', 'price', 'brand', 'etat','url_advert','promoted', 'taille', 'favourite_count', 'photo_advert', 'date_advert',
                             'description_advert', 'idUser','login','email', 'gender', 'item_count', 'followers_count', 'following_count',
                     'feedback_count', 'positive_feedback_count', 'neutral_feedback_count',
                     'negative_feedback_count', 'account_ban_date', 'is_account_ban_permanent',
                     'is_publish_photos_agreed', 'expose_location', 'city', 'country_title', 'business_account',
                     'total_items_count', 'verification_email', 'verification_facebook', 'verification_google',
                     'verification_phone', 'is_hated', 'hates_you', 'can_view_profile', 'item_titles_advice',
                     'ratings_advice', 'feedbacks_advice', 'item_id_dressing', 'item_title_dressing',
                     'item_brand_dressing', 'item_size_dressing', 'item_price_dressing', 'item_created_dressing',
                     'item_description_dressing', 'photo'])

In [17]:
s=vinted_session()
legend='Adidas+campus+00s+chaussure'
dataset_final=scrapping_annonces(dataset_final,legend,s)

Une erreur s'est produite : local variable 'annonce_date' referenced before assignment
Une erreur s'est produite lors du scraping d'une annonce : cannot unpack non-iterable NoneType object
Une erreur s'est produite : local variable 'annonce_date' referenced before assignment
Une erreur s'est produite lors du scraping d'une annonce : cannot unpack non-iterable NoneType object
Une erreur s'est produite : local variable 'annonce_date' referenced before assignment
Une erreur s'est produite lors du scraping d'une annonce : cannot unpack non-iterable NoneType object
Une erreur s'est produite : local variable 'annonce_date' referenced before assignment
Une erreur s'est produite lors du scraping d'une annonce : cannot unpack non-iterable NoneType object
Une erreur s'est produite : 429 Client Error: Too Many Requests for url: https://www.vinted.fr/api/v2/users/58583056/items?page=1&per_page=6&order=relevance
Une erreur s'est produite lors du scraping d'une annonce : cannot unpack non-iterable N

In [18]:
dataset_final.head()

,id_advert,title_advert,price,brand,etat,url_advert,promoted,taille,favourite_count,photo_advert,...,ratings_advice,feedbacks_advice,item_id_dressing,item_title_dressing,item_brand_dressing,item_size_dressing,item_price_dressing,item_created_dressing,item_description_dressing,photo
0,3983113025,Adidas Campus 00s Black 39,49.9,adidas Superstar,Neuf avec étiquette,https://www.vinted.fr/items/3983113025-adidas-...,False,39,9,https://images1.vinted.net/t/01_00f47_pfAtSsRv...,...,[],[],"[3983120599, 3983119524, 3983119392, 398311862...","[Adidas Campus 00s Black 37, Adidas Campus 00s...","[adidas Superstar, adidas Superstar, adidas Su...","[37, 36, 44, 43, 42, 41, 40, 42, 41, 43, 38, 4...","[49.9, 49.9, 49.9, 49.9, 49.9, 49.9, 49.9, 49....","[2024-01-15T17:27:33+01:00, 2024-01-15T17:27:1...","[Neuf et original, avec boîte 📦, livraison sou...",Null
1,3980586104,Adidas Campus 00s black 37,50.0,adidas Superstar,Neuf avec étiquette,https://www.vinted.fr/items/3980586104-adidas-...,False,37,20,https://images1.vinted.net/t/02_00540_LTeTsx3h...,...,[],[],"[3977749044, 3979106891, 3980091518]","[ Adidas Campus 00s black 39, Adidas Campus 00...","[adidas Superstar, adidas Superstar, adidas Su...","[39, 38, 40]","[50.0, 50.0, 50.0]","[2024-01-14T15:24:13+01:00, 2024-01-14T18:10:3...","[Chaussures innovantes originales neuves, inut...",https://images1.vinted.net/t/01_008c2_qYxPTBWm...
2,3982750602,Adidas Campus gray 00s 38,65.0,adidas Superstar,Neuf avec étiquette,https://www.vinted.fr/items/3982750602-adidas-...,False,38,15,https://images1.vinted.net/t/01_01b31_8Qo2pPUJ...,...,[],[],"[3982752074, 3982747240]","[ Adidas Campus gray 00s 39, Adidas Campus gr...","[adidas Superstar, adidas Superstar]","[39, 37]","[65.0, 65.0]","[2024-01-15T16:11:03+01:00, 2024-01-15T16:10:0...","[Les chaussures sont originales, neuves et inu...",https://images1.vinted.net/t/03_00ab6_zyDqB5Qf...
3,3977749044,Adidas Campus 00s black 39,50.0,adidas Superstar,Neuf avec étiquette,https://www.vinted.fr/items/3977749044-adidas-...,False,39,27,https://images1.vinted.net/t/03_008d8_2Hozioez...,...,[],[],"[3979106891, 3980586104, 3980091518]","[Adidas Campus 00s black 38, Adidas Campus 00s...","[adidas Superstar, adidas Superstar, adidas Su...","[38, 37, 40]","[50.0, 50.0, 50.0]","[2024-01-14T18:10:39+01:00, 2024-01-14T23:06:1...","[Chaussures innovantes originales neuves, inut...",https://images1.vinted.net/t/01_008c2_qYxPTBWm...
4,3983163202,Adidas Campus 00s (Women's) Black Eu 38,80.0,adidas Originals,Neuf avec étiquette,https://www.vinted.fr/items/3983163202-adidas-...,False,38,10,https://images1.vinted.net/t/01_0256a_b54KKpX8...,...,"[5, 5, 5]","[Très bien, Évaluation automatique : vente réa...",[],[],[],[],[],[],[],https://images1.vinted.net/t/02_0020b_hdyRNXh7...


In [27]:
dataset_final.to_csv('dataset_final.csv')

In [ ]:
def scrapping_other_informations(login, s):
    id_user,feedback_reputation=[],[]
    try:
        for i in login:
            url = f"https://www.vinted.fr/api/v2/users/{i}?localize=false"
            check = s.get(url)
            check.raise_for_status()
            json_data = json.loads(check.text)
            id_user.append(i)
            feedback_reputation.append(json_data['user']['feedback_reputation'])

        df = pd.DataFrame({'idUser': id_user, 'feedback_reputation':feedback_reputation})   
        return df

    except Exception as e:
        print(f"Une erreur s'est produite lors du scraping des annonces : {e}")
        return df